In [ ]:
!pip install geopandas
!pip install contextily

In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [2]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import pandas as pd
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [3]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [61]:
#iso3 = "GHA"
iso3 = "UGA"
file_path = f"/projects/my-public-bucket/AOIs/vero_1deg_tileindex/tileindex_{iso3}.csv"
tileindex_df = pd.read_csv(file_path)
# Display the first few rows of the DataFrame
#print(tileindex_df.head())

In [62]:
json_files = tileindex_df['s3path']
AOIs = json_files
print(len(AOIs))
aoi_name = os.path.basename(AOIs[0]).split('.')[0]
aoi_name

34


'tile_num_37710'

In [63]:
AOIs[1]

'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_37711.geojson'

In [64]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

#PRODUCTS = ["L4A"]  #To run for only 1 or 2 products, change "PRODUCTS"
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
#PRODUCTS = ["L2B"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files
#AOIs = json_files[0:10]
#AOIs = json_files[10:20]
#AOIs = json_files[20:30]
#AOIs = json_files[30:35]

for each_aoi in AOIs: 
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh98,geolocation/sensitivity_a2",
           query="quality_flag == 1",
           limit = 75_000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            #queue="maap-dps-worker-32vcpu-64gb",
            queue="maap-dps-worker-64gb",
            username="leitoldv",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result


Thanks, I'll run the GEDI L2A subsetter for tile_num_37710!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37711!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37712!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37887!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37888!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37889!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37890!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37891!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_37892!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38067!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38068!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38069!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38070!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38071!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38072!!
Thanks, I'll run the GEDI L2A subsetter for tile_num_38247!!
Thanks, I'll run the GED

In [67]:
PRODUCTS = ["L2A"] #,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"
## if necessary ## re-run failed subsets
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_22771.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_32122.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_38249.geojson'
##UGA
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_37887.geojson'
#each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_38068.geojson'
each_aoi = 'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/vero_1deg_tiles_52N52S_land/tile_num_38069.geojson'

inputs = dict(
    aoi=each_aoi,
    doi="L2A",
    lat="lat_lowestmode",
    lon="lon_lowestmode",
    beams="all",
    #columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80",
    columns="shot_number,lon_lowestmode,lat_lowestmode,rh98,geolocation/sensitivity_a2",
    query="quality_flag == 1",
    limit = 75_000,
    #temporal="-",
    output=f"{os.path.basename(each_aoi).split('.')[0]}_L2A.gpkg"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset",
    version="0.9.0",
    queue="maap-dps-worker-32vcpu-64gb",
    username="leitoldv",
    **inputs
)
inputs
job_id = result.id
job_id or result

'2d741c58-3b92-4032-9e49-66402bd1ac34'

In [68]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


999


In [69]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2025/01/10"  # Can now set name
out_dir = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("L2A.gpkg"):
            source_path = os.path.join(dirName, fname)
            #print(source_path)
            target_path = os.path.join(out_dir, fname)
            #print(target_path)
#            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
            print(f"Copying {fname}...")
            subprocess.call(['cp', source_path, target_path])
#            else:
#                print(f"{fname} already exists in the target directory. Skipping...")

Copying tile_num_37712_L2A.gpkg...
Copying tile_num_38432_L2A.gpkg...
Copying tile_num_38252_L2A.gpkg...
Copying tile_num_37892_L2A.gpkg...
Copying tile_num_38072_L2A.gpkg...
Copying tile_num_31944_L2A.gpkg...
Copying tile_num_31945_L2A.gpkg...
Copying tile_num_31763_L2A.gpkg...
Copying tile_num_31942_L2A.gpkg...
Copying tile_num_31943_L2A.gpkg...
Copying tile_num_31946_L2A.gpkg...
Copying tile_num_31941_L2A.gpkg...
Copying tile_num_31764_L2A.gpkg...
Copying tile_num_31940_L2A.gpkg...
Copying tile_num_31765_L2A.gpkg...
Copying tile_num_32124_L2A.gpkg...
Copying tile_num_32123_L2A.gpkg...
Copying tile_num_32125_L2A.gpkg...
Copying tile_num_32299_L2A.gpkg...
Copying tile_num_32300_L2A.gpkg...
Copying tile_num_32300_L2A.gpkg...
Copying tile_num_32121_L2A.gpkg...
Copying tile_num_32119_L2A.gpkg...
Copying tile_num_32299_L2A.gpkg...
Copying tile_num_32126_L2A.gpkg...
Copying tile_num_32122_L2A.gpkg...
Copying tile_num_32120_L2A.gpkg...
Copying tile_num_32120_L2A.gpkg...
Copying tile_num_324

In [70]:
# Define the folder path containing the output files from subsetting of tiles
folder_path = "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_gedi_L2A_tiles"

# Create a list to store the GeoPackage file paths
gpkg_files = []

# Use glob to find all .gpkg files in the folder
pattern1 = os.path.join(folder_path, '*.gpkg')
gpkg_files = glob.glob(pattern1)
print(len(gpkg_files))


1002
